In [1]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving.git

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45753, done.
remote: Counting objects: 100% (11663/11663), done.
remote: Compressing objects: 100% (11648/11648), done.
remote: Total 45753 (delta 26), reused 11639 (delta 14), pack-reused 34090
Receiving objects: 100% (45753/45753), 2.16 GiB | 43.03 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (45592/45592), done.


In [2]:
!mv "/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt" "/content/Omdena-Quantum-Self-Driving/Images/"

In [ ]:
# !du -h /content/Omdena-Quantum-Self-Driving

In [3]:
from struct import unpack
from tqdm import tqdm
import os

img_dir = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
data_file = "/content/Omdena-Quantum-Self-Driving/Images/data.txt"

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

In [7]:
def build_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = True

    flatten_output = Flatten()(base_model.output)

    angle_input = Input(shape=(1,))
    fused_output = Concatenate()([flatten_output, angle_input])

    fc1 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(fused_output)
    dropout1 = Dropout(0.2)(fc1)
    fc2 = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(fc1)
    output = Dense(1)(fc2)

    model = Model(inputs=[base_model.input, angle_input], outputs=output)

    return model


# Load and preprocess a single image
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg19.preprocess_input(image)
    return image

def create_dataset(image_files, angles):
    image_paths = [os.path.join(img_dir, file_name) for file_name in image_files]
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    angle_dataset = tf.data.Dataset.from_tensor_slices(angles)
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))
    return dataset

# Get the list of image files
image_files = []

# Load the steering angles
angles_dict = {}
with open(data_file, 'r') as file:
    for line in file:
        img_file, angle = line.strip().split()
        image_files.append(img_file)
        angles_dict[img_file] = float(angle)

# Get the angles for the image files
angles = [angles_dict[file_name] for file_name in image_files]

# Split the image files into training and validation sets
validation_split = 0.2
num_validation_samples = int(validation_split * len(image_files))
train_image_files = image_files[num_validation_samples:]
train_image_angles = angles[num_validation_samples:]
val_image_files = image_files[:num_validation_samples]
val_image_angles = angles[:num_validation_samples]

# Create the training and validation datasets with both inputs
train_dataset = create_dataset(train_image_files, train_image_angles)
val_dataset = create_dataset(val_image_files, val_image_angles)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

# Define the input shape for the angle dataset
angle_input_shape = (1,)

# Create the VGG-16 model
model = build_model()

# Define the mean squared error as the loss function
def mse_loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=mse_loss)

# Modify the fit call to pass both inputs
train_dataset = train_dataset.map(lambda img, angle: ((img, angle), angle))
val_dataset = val_dataset.map(lambda img, angle: ((img, angle), angle))

In [ ]:
# Train the model
epochs = 50
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

Epoch 1/50
1136/1136 [==============================] - 533s 463ms/step - loss: 111.2662 - val_loss: 2459.2766
Epoch 2/50
1136/1136 [==============================] - 525s 459ms/step - loss: 47.2643 - val_loss: 2370.3228
Epoch 3/50
1136/1136 [==============================] - 523s 457ms/step - loss: 33.1392 - val_loss: 2320.9272
Epoch 4/50
1136/1136 [==============================] - 521s 456ms/step - loss: 30.0409 - val_loss: 2167.6414
Epoch 5/50
1136/1136 [==============================] - 521s 455ms/step - loss: 25.8302 - val_loss: 1935.6454
Epoch 6/50
1136/1136 [==============================] - 524s 457ms/step - loss: 24.7835 - val_loss: 1719.2152
Epoch 7/50
1136/1136 [==============================] - 522s 457ms/step - loss: 20.7051 - val_loss: 1603.2896
Epoch 8/50
1136/1136 [==============================] - 521s 456ms/step - loss: 18.3696 - val_loss: 1328.1682
Epoch 9/50
1136/1136 [==============================] - 523s 457ms/step - loss: 16.0950 - val_loss: 1159.1401
Epoch 10/

In [ ]:

marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
               raise TypeError("issue reading jpeg file")


bads = []

for dirName, subdirList, fileList in os.walk(img_dir):
    imagesList = fileList
    for img in tqdm(imagesList):
      image = os.path.join(img_dir,img)
      image = JPEG(image)
      try:
        image.decode()
      except:
        bads.append(img)


for name in bads:
  os.remove(os.path.join(img_dir,name))

100%|██████████| 45568/45568 [00:03<00:00, 13119.73it/s]
